Data Reading

In [3]:
import pandas as pd

movies = pd.read_csv("../dataset/movie.csv")
ratings = pd.read_csv("../dataset/rating.csv")

print(movies.head())
print(ratings.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating            timestamp
0       1        2     3.5  2005-04-02 23:53:47
1       1       29     3.5  2005-04-02 23:31:16
2       1       32     3.5  2005-04-02 23:33:39
3       1       47     3.5  2005-04-02 23:32:07
4       1       50     3.5  2005-04-02 23:29:40


Preprocessing

Merge datasets
Clean genres and titles
Compute average rating per movie
Create a genre + rating based profile

In [4]:
# Merge average ratings
avg_ratings = ratings.groupby("movieId")["rating"].mean().reset_index()
avg_ratings.columns = ["movieId", "avg_rating"]

movies = movies.merge(avg_ratings, on="movieId", how="left")

# Replace '|' with space in genres
movies["genres"] = movies["genres"].str.replace("|", " ", regex=False)